In [14]:
from ultralytics import YOLO
import cv2
import numpy as np
from google.colab import drive
drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [26]:
model = YOLO('yolov8n.pt')

In [16]:
# All class names for detections
class_names = model.names
# Random colors for bounding boxes
BBOX_COLORS = np.random.randint(0,255,size=(len(class_names), 3), dtype='uint8')

In [ ]:
video_path = '/content/gdrive/MyDrive/Projects/MultiDetection/person02.mp4'
thresh=0.5
out_path = f'{video_path}_output.mp4'
video_cap = cv2.VideoCapture(video_path)
ret, frame = video_cap.read()
H, W ,_ = frame.shape
output = cv2.VideoWriter(out_path, cv2.VideoWriter_fourcc(*'MP4V'), int(video_cap.get(cv2.CAP_PROP_FPS)),(W,H))
while ret:
  results = model.track(frame, stream=True)
  for detection in results:
    classes_names = detection.names
    for bbox in detection.boxes:
      if bbox.conf[0] > thresh:
        [x1,y1,x2,y2] = bbox.xyxy[0]
        x1,y1,x2,y2 = int(x1), int(y1), int(x2), int(y2)
        class_ = int(bbox.cls[0])
        class_name = classes_names[class_]
        color = (int(BBOX_COLORS[class_][0]),int(BBOX_COLORS[class_][1]),int(BBOX_COLORS[class_][2]))
        cv2.rectangle(frame, (x1,y1), (x2,y2), color, 5)
        cv2.putText(frame, f'{classes_names[int(class_)]} {bbox.conf[0]*100:.2f}%',
                    (x1,y1), cv2.FONT_HERSHEY_SIMPLEX,5,color,5)
  output.write(frame)
  ret, frame=video_cap.read()
video_cap.release()
output.release()
cv2.destroyAllWindows()